In [1]:
import openpyxl
import os
import shutil
from PyPDF2 import PdfReader
import cv2

In [2]:
with open("example.json","r") as f:
    data = eval(f.read())

In [3]:

bix_name=data.get('business_name')
#Create Directory For Client Files
try:
    directory = os.path.join(os.getcwd(),'ClientFiles',bix_name)
    if os.path.exists(directory):
        
        response = input("Client name already found in directory. Do you wish to overwrite this folder? (y/N): ")
        
        if response and response.lower()[0] == "y":
            shutil.rmtree(directory)
            os.mkdir(directory)
        else:
            print('No Override')
    else:
        os.mkdir(directory)
except Exception as e:
    print(e)

    #raise FileExistsError('Organization By This Name Already Exists. Check If Client Is Duplicate, If Not, add current date to client name')

No Override


In [4]:
template_file  = "Employee Retention Credit Calculator .xlsx"
workbook = openpyxl.load_workbook(template_file)
workbook.sheetnames

['Version features',
 '01_P&L',
 '02_941',
 '03_Payroll',
 '04_Health',
 '05_PPP',
 '06_Owners',
 '07_Govt Shutdown',
 'PPP instr',
 'Common EEs',
 'Q2_2020 941X',
 'Q3_2020 941X',
 'Q4_2020 941X',
 'Q1_2021 941X',
 'Q2_2021 941X',
 'Q3_2021 941X',
 'Q4_2021 941X',
 'Sub Rpt 1',
 'Sub Rpt 2',
 'Sub Rpt 3',
 '08_Calculator']

In [5]:
# Separate Raw Request By Section
profit_loss = {}
gov_shutdown = {}
# payroll = List
# healthcare = List
# ppploan = List
# TODO convert to case switch
for key, value in data.items():
    if "gov" in key:
        gs_key = key.split("_", 2)
        gov_shutdown[gs_key[-1]] = value
    elif "pl" in key:
        profit_loss[key] = value
    elif key == "owner_calculation":
        owner_calculations = value
        
    elif key == "payroll_documents":
        payroll = value
    elif key == "ppp_info":
        ppploan = value
    elif key == "doc_941":
        doc_941 = value
    elif key == "hp_contributions":
        healthcare = value
    elif key == 'business_name':
        business_name = value

sub = {'profit_loss': profit_loss,
       'gov_shutdown': gov_shutdown,
       'owner_calculations': owner_calculations,
       'payroll_documents': payroll,
       'healthcare': healthcare,
       'doc941': doc_941,
       'ppp_loan': ppploan,
       'business_name':business_name}


#Split Profit/Loss
p_l = sub.get('profit_loss')
for key, value in p_l.items():
    p_l[key] = [{x.split("-")[0]:x.split("-")[-1]} for x in value.split(";")]
#     #p_l[key] = [{x.split("-")[0]:x.split("-")[-1]} for x in value]
#     #print([{x.split("-")[0]:x.split("-")[1]} for x in value])
sub['profit_loss'] = p_l
sub

#Split Gov Shutdown
g_s = sub.get("gov_shutdown")
for key, value in g_s.items():
    g_s[key] = value.split(";") if ";" in value else []
sub['gov_shutdown'] = g_s

# Parse Resource Ids for payroll documents
#TODO copy for healthcare documents because they should also be submited per quarter
pr = sub.get("payroll_documents")
payroll_ids = []
for attachment in pr:
    pq = attachment.get('payroll_quarter')
    resource_id = attachment.get('docs').pop().split('/')[-1]
    payroll_ids.append({'quarter':pq,'resource_id':resource_id})
sub['payroll_documents'] = payroll_ids


sub['doc941']=sub.get('doc941').pop().split('/')[-1]


In [6]:
#Input Profit And Loss Values into first tab
sheet = workbook['01_P&L']
"""
Q1-2019: B14
Q2-2019: B17
Q3-2019: B20
Q4-2019: B23
Q1-2020: E14
Q2-2020: E17
Q3-2020: E20
Q4-2020: E23
Q1-2021: H14
Q2-2021: H17
Q3-2021: H20
"""
XPLTRANSLATION = {
"Q1-2019": "B14",
"Q2-2019": "B17",
"Q3-2019": "B20",
"Q4-2019": "B23",
"Q1-2020": "E14",
"Q2-2020": "E17",
"Q3-2020": "E20",
"Q4-2020": "E23",
"Q1-2021": "H14",
"Q2-2021": "H17",
"Q3-2021": "H20"
}
for key,value in sub.get('profit_loss').items():
    year = key.split("_")[-1]
    for entry in value:
        for quarter, p_l in entry.items():
            #print(year,quarter,p_l)
            SEARCH_KEY = f"{quarter}-{year}"
            destinationCell = XPLTRANSLATION.get(SEARCH_KEY)
            #print(destinationCell)
            if destinationCell:
                sheet[destinationCell].value = p_l
            else:
                continue

#workbook.save(f"{directory}/Test_Output.xlsx")


In [7]:
TESTDOC = "TestClients/Test Client 1/20_21 941 FORMS.PDF"
TESTTABULAR = "TestClients/Test Client 1/fig-tree-psychotherapy-center-payroll-summary-2020-Q2.pdf"
TIFF_IMAGE = '0Obj4.tiff'
reader = PdfReader(TESTDOC)
#print(reader.pages[0].images[0].data)

from PIL import Image
import pytesseract
import numpy as np
pytesseract.pytesseract.tesseract_cmd= r'C:\Program Files\Tesseract-OCR\tesseract'
img1 = np.array(Image.open(TIFF_IMAGE))
text = pytesseract.image_to_string(Image.open(TIFF_IMAGE))

In [8]:
image = cv2.imread('0Obj4.tiff')
results = pytesseract.image_to_data(image,output_type=pytesseract.Output.DICT)
for i in range(0, len(results["text"])):
   x = results["left"][i]
   y = results["top"][i]

   w = results["width"][i]
   h = results["height"][i]

   text = results["text"][i]
   conf = int(results["conf"][i])

   if conf > 75:
       text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
       cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
       cv2.putText(image, text, (x, y - 10), 
cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 200), 2)

In [9]:
cv2.imwrite('test.png',image)

True

In [11]:
# from pdf2image import convert_from_path
# pages = convert_from_path(r'TestClients/Test Client 1/20_21 941 FORMS.PDF')
# for i,pg in enumerate(pages):
#     pg.save(f"{i}-out.png",'PNG')

import fitz  # PyMuPDF, imported as fitz for backward compatibility reasons
file_path = "TestClients/Test Client 1/20_21 941 FORMS.PDF"
doc = fitz.open(file_path)  # open document
for i, page in enumerate(doc):
    pix = page.get_pixmap()  # render page to an image
    pix.save(f"page_{i}.png")

ModuleNotFoundError: No module named 'tools'

In [ ]:
#Parse Healthcare Documents

In [ ]:
with open("test.pdf","rb") as f:
    content= f.read()